In [2]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.9 MB/s eta 0:00:00


In [18]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader

from torchvision.utils import save_image
from torchsummary import summary

import matplotlib.pyplot as plt
import seaborn as sns
from torch_geometric.data import Data, DataLoader


In [19]:
if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print("GPU")
else:
  device = torch.device("cpu")
  print("CPU")

CPU


### Import Cora dataset

In [106]:
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx

dataset = Planetoid(root='data', name='Cora')
data = dataset[0].to(device)

print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Number of features: {data.x.size(1)}')
print(f'Number of classes: {dataset.num_classes}')


Number of nodes: 2708
Number of nodes: 10556
Number of features: 1433
Number of classes: 7
Has isolated nodes: False
Is undirected: True
Average node degree: 3.90


### Approach for neighborhood aggregation

- $h^0_v = x_v$
- $h^{l+1}_v = σ(W_l ∑_{u \in N(v)} \frac{h^{(l)}_v}{|N(v)|}) + B_lh^{(l)}_v, \forall l \in {0, ..., L-1}$
- $z_v = h^{(L)}_v$

Notation:
1. $h^{(l)}_v$: the hidden representation of node v at layer l
2. $W_k$ : Weight matrix for neighborhood aggregation
3. $B_k$ : Bias matrix for transforming hidden vector of self


In [148]:

class GraphConvolution(nn.Module):
  def __init__(self, input_size, output_size):
    super(GraphConvolution, self).__init__()
    self.W = nn.Parameter(torch.rand(input_size, output_size))
    self.B = nn.Parameter(torch.rand(input_size, output_size))

  def forward(self, x, A):
    H_W = torch.matmul(x, self.W)
    H_B = torch.matmul(x, self.B)
    H = torch.matmul(A, H_W) + H_B
    return H


class GCN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(GCN, self).__init__()
    self.conv1 = GraphConvolution(input_size, hidden_size)
    self.conv2 = GraphConvolution(hidden_size, hidden_size)

  def forward(self, x,A):
    x = F.relu(self.conv1(x, A))
    x = F.relu(self.conv2(x, A))
    return x



In [151]:
G = to_networkx(data, to_undirected=True)

edge_index = data.edge_index
num_nodes = data.num_nodes

# Adjacency Matrix: A
A = torch.zeros((num_nodes, num_nodes), dtype=torch.float)
A[edge_index[0], edge_index[1]] = 1.0
A[edge_index[1], edge_index[0]] = 1.0


input_dim = dataset.num_features
hidden_dim = 128
output_dim = dataset.num_classes
gcn_model = GCN(input_dim, hidden_dim, output_dim)
print(gcn_model)

lr = 1e-2
num_epochs = 100

CE_Loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(gcn_model.parameters(), lr=lr)

for epoch in range(num_epochs):
  optimizer.zero_grad()

  output = gcn_model.forward(data.x, A)

  loss = CE_Loss(output[data.train_mask], data.y[data.train_mask])

  loss.backward()
  optimizer.step()

  print(f'Epoch {epoch + 1}/{num_epochs}, Cross Entropy Loss: {loss.item():.2f}')


gcn_model.eval()
with torch.no_grad():
  pred = gcn_model(data.x, A)
  pred = pred.argmax(dim=1)
  accuracy = (pred[data.test_mask] == data.y[data.test_mask]).sum().item() / len(pred[data.test_mask])

print(f'Test Accuracy: {accuracy * 100:.2f}%')

GCN(
  (conv1): GraphConvolution()
  (conv2): GraphConvolution()
)
Epoch 1/100, Cross Entropy Loss: 3869.09
Epoch 2/100, Cross Entropy Loss: 2676.53
Epoch 3/100, Cross Entropy Loss: 1501.66
Epoch 4/100, Cross Entropy Loss: 853.98
Epoch 5/100, Cross Entropy Loss: 328.41
Epoch 6/100, Cross Entropy Loss: 349.67
Epoch 7/100, Cross Entropy Loss: 428.97
Epoch 8/100, Cross Entropy Loss: 667.65
Epoch 9/100, Cross Entropy Loss: 827.46
Epoch 10/100, Cross Entropy Loss: 607.56
Epoch 11/100, Cross Entropy Loss: 502.43
Epoch 12/100, Cross Entropy Loss: 293.17
Epoch 13/100, Cross Entropy Loss: 262.40
Epoch 14/100, Cross Entropy Loss: 283.28
Epoch 15/100, Cross Entropy Loss: 305.64
Epoch 16/100, Cross Entropy Loss: 263.07
Epoch 17/100, Cross Entropy Loss: 168.67
Epoch 18/100, Cross Entropy Loss: 109.96
Epoch 19/100, Cross Entropy Loss: 131.24
Epoch 20/100, Cross Entropy Loss: 105.93
Epoch 21/100, Cross Entropy Loss: 50.48
Epoch 22/100, Cross Entropy Loss: 37.52
Epoch 23/100, Cross Entropy Loss: 46.38

In [159]:
from torch_geometric.nn import GCNConv

class built_GCN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(built_GCN, self).__init__()
    self.conv1 = GCNConv(input_size, hidden_size)
    self.conv2 = GCNConv(hidden_size, output_size)

  def forward(self, data):
    node, edge = data.x, data.edge_index
    node = self.conv1(node, edge_index)
    node = F.relu(node)
    node = self.conv2(node, edge_index)
    return F.relu(node)


In [160]:

edge_index = data.edge_index
num_nodes = data.num_nodes

input_dim = dataset.num_features
hidden_dim = 64
output_dim = dataset.num_classes

built_gcn_model = built_GCN(input_dim, hidden_dim, output_dim)
print(built_gcn_model)

lr = 1e-2
num_epochs = 100

CE_Loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(built_gcn_model.parameters(), lr=lr)

for epoch in range(num_epochs):
  optimizer.zero_grad()

  output = built_gcn_model(data)

  loss = CE_Loss(output[data.train_mask], data.y[data.train_mask])

  loss.backward()
  optimizer.step()

  print(f'Epoch {epoch + 1}/{num_epochs}, Cross Entropy Loss: {loss.item():.2f}')


built_gcn_model.eval()
with torch.no_grad():
    pred = built_gcn_model(data)
    pred = pred.argmax(dim=1)
    accuracy = (pred[data.test_mask] == data.y[data.test_mask]).sum().item() / len(pred[data.test_mask])

print(f'Test Accuracy: {accuracy * 100:.2f}%')

built_GCN(
  (conv1): GCNConv(1433, 64)
  (conv2): GCNConv(64, 7)
)
Epoch 1/100, Cross Entropy Loss: 1.94
Epoch 2/100, Cross Entropy Loss: 1.71
Epoch 3/100, Cross Entropy Loss: 1.43
Epoch 4/100, Cross Entropy Loss: 1.15
Epoch 5/100, Cross Entropy Loss: 0.90
Epoch 6/100, Cross Entropy Loss: 0.67
Epoch 7/100, Cross Entropy Loss: 0.48
Epoch 8/100, Cross Entropy Loss: 0.34
Epoch 9/100, Cross Entropy Loss: 0.24
Epoch 10/100, Cross Entropy Loss: 0.16
Epoch 11/100, Cross Entropy Loss: 0.11
Epoch 12/100, Cross Entropy Loss: 0.08
Epoch 13/100, Cross Entropy Loss: 0.05
Epoch 14/100, Cross Entropy Loss: 0.04
Epoch 15/100, Cross Entropy Loss: 0.02
Epoch 16/100, Cross Entropy Loss: 0.02
Epoch 17/100, Cross Entropy Loss: 0.01
Epoch 18/100, Cross Entropy Loss: 0.01
Epoch 19/100, Cross Entropy Loss: 0.01
Epoch 20/100, Cross Entropy Loss: 0.01
Epoch 21/100, Cross Entropy Loss: 0.00
Epoch 22/100, Cross Entropy Loss: 0.00
Epoch 23/100, Cross Entropy Loss: 0.00
Epoch 24/100, Cross Entropy Loss: 0.00
Epoch